In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [3]:
# INITIALIZING SQLLITE
import sqlite3

DATABASE_PATH = './data/sf/database.sqlite'
conn = sqlite3.connect(DATABASE_PATH)
cursor = conn.cursor()

In [4]:
def run_query(query):
    cursor.execute(query)
    return cursor.fetchall()

def run_pd_query(query):
    return pd.read_sql(query, conn)

### Taking a closer look at our database

In [5]:
# SQL query to obtain all of the weather information
WEATHER_QUERY = 'SELECT * FROM weather;'
weather_df = run_pd_query(WEATHER_QUERY)

In [6]:
# SQL query to create a row for each trip and the start and end station
TRIP_STATION_QUERY = 'SELECT * FROM trip'
sf_df = run_pd_query(TRIP_STATION_QUERY)


In [7]:
# remove any rows that have empty columns
sf_df = sf_df.dropna(how="any")
weather_df = weather_df.dropna(how="any")

In [8]:
# remove the outliers based on duration
z_scores = stats.zscore(sf_df["duration"])

abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3)
sf_df = sf_df[filtered_entries]

In [9]:
# convert the start and end dates to pandas datetime
sf_df["start_date"] = pd.to_datetime(sf_df["start_date"], format='%m/%d/%Y %H:%M')
sf_df["end_date"] = pd.to_datetime(sf_df["end_date"], format='%m/%d/%Y %H:%M')

weather_df["date"] = pd.to_datetime(weather_df["date"], format='%m/%d/%Y')

# add month column
sf_df["month"] = sf_df["start_date"].dt.month
sf_df["end_month"] = sf_df["end_date"].dt.month
weather_df["month"] = weather_df["date"].dt.month

# add year column
sf_df["year"] = sf_df["start_date"].dt.year
weather_df["year"] = weather_df["date"].dt.year


# add day column
sf_df["day"] = sf_df["start_date"].dt.day
sf_df["day_of_year"] = sf_df["start_date"].dt.day_of_year
weather_df["day"] = weather_df["date"].dt.day


# add hour column
sf_df["hour"] = sf_df["start_date"].dt.hour

# make duration into minutes
sf_df["duration_min"] = sf_df["duration"] / 60

In [10]:
sf_df.head()

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,month,end_month,year,day,day_of_year,hour,duration_min
0,4069,174,2013-08-29 09:08:00,2nd at South Park,64,2013-08-29 09:11:00,2nd at South Park,64,288,Subscriber,94114,8,8,2013,29,241,9,2.900000
1,4073,1067,2013-08-29 09:24:00,South Van Ness at Market,66,2013-08-29 09:42:00,San Francisco Caltrain 2 (330 Townsend),69,321,Subscriber,94703,8,8,2013,29,241,9,17.783333
2,4074,1131,2013-08-29 09:24:00,South Van Ness at Market,66,2013-08-29 09:43:00,San Francisco Caltrain 2 (330 Townsend),69,317,Subscriber,94115,8,8,2013,29,241,9,18.850000
3,4075,1117,2013-08-29 09:24:00,South Van Ness at Market,66,2013-08-29 09:43:00,San Francisco Caltrain 2 (330 Townsend),69,316,Subscriber,94122,8,8,2013,29,241,9,18.616667
4,4076,1118,2013-08-29 09:25:00,South Van Ness at Market,66,2013-08-29 09:43:00,San Francisco Caltrain 2 (330 Townsend),69,322,Subscriber,94597,8,8,2013,29,241,9,18.633333


In [11]:
weather_df.head()

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code,month,year,day
0,2013-08-29,74,68,61,61,58,56,93,75,57,...,11,28,0,4,,286,94107,8,2013,29
1,2013-08-30,78,69,60,61,58,56,90,70,50,...,13,35,0,2,,291,94107,8,2013,30
2,2013-08-31,71,64,57,57,56,54,93,75,57,...,15,31,0,4,,284,94107,8,2013,31
3,2013-09-01,74,66,58,60,56,53,87,68,49,...,13,29,0,4,,284,94107,9,2013,1
4,2013-09-02,75,69,62,61,60,58,93,77,61,...,12,30,0,6,,277,94107,9,2013,2


In [12]:
weather_df.columns

Index(['date', 'max_temperature_f', 'mean_temperature_f', 'min_temperature_f',
       'max_dew_point_f', 'mean_dew_point_f', 'min_dew_point_f',
       'max_humidity', 'mean_humidity', 'min_humidity',
       'max_sea_level_pressure_inches', 'mean_sea_level_pressure_inches',
       'min_sea_level_pressure_inches', 'max_visibility_miles',
       'mean_visibility_miles', 'min_visibility_miles', 'max_wind_Speed_mph',
       'mean_wind_speed_mph', 'max_gust_speed_mph', 'precipitation_inches',
       'cloud_cover', 'events', 'wind_dir_degrees', 'zip_code', 'month',
       'year', 'day'],
      dtype='object')

In [13]:
float_cols = ['max_temperature_f', 'mean_temperature_f', 'min_temperature_f',
       'max_dew_point_f', 'mean_dew_point_f', 'min_dew_point_f',
       'max_humidity', 'mean_humidity', 'min_humidity',
       'max_sea_level_pressure_inches', 'mean_sea_level_pressure_inches',
       'min_sea_level_pressure_inches', 'max_visibility_miles',
       'mean_visibility_miles', 'min_visibility_miles', 'max_wind_Speed_mph',
       'mean_wind_speed_mph', 'max_gust_speed_mph', 'precipitation_inches',
       'cloud_cover', 'wind_dir_degrees']
weather_df[float_cols] = weather_df[float_cols].apply(pd.to_numeric, errors="coerce")

In [14]:
weather_df = weather_df.dropna()

In [15]:
# sf_usage = sf_df.groupby(["month", "day", "start_station_id"]).agg(usage=('id', 'count')).reset_index(drop=False)
# sf_usage['ewm'] = sf_usage.groupby(['start_station_id'])[["usage"]].transform(lambda x: x.ewm(halflife=7).mean())


In [93]:
sf_usage = (
    sf_df.groupby(["month", "day", "year", "start_station_id"]).agg(
        usage=('id', 'count'),
        date=("start_date", "min"),
    )
).reset_index(drop=False)
weather_aggregated = (
    weather_df.drop("date", axis=1).groupby(["month", "day", "year"]).agg({i: "mean" for i in float_cols}).reset_index()
)
sf_usage = sf_usage.merge(weather_aggregated, on=["month", "day", "year"])
sf_usage["is_weekend"] = sf_usage["date"].dt.day_of_week > 5
# lag not currently used
sf_usage["usage_lag"] = sf_usage["usage"].shift(1)
sf_usage.dropna()

,month,day,year,start_station_id,usage,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,...,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,wind_dir_degrees,is_weekend,usage_lag,usage_lag2
2,1,1,2014,5,1,2014-01-01 16:31:00,59.0,48.0,36.50,41.75,...,6.25,9.75,1.25,11.5,0.0,1.25,324.0,False,1.0,1.0
3,1,1,2014,7,2,2014-01-01 10:50:00,59.0,48.0,36.50,41.75,...,6.25,9.75,1.25,11.5,0.0,1.25,324.0,False,1.0,1.0
4,1,1,2014,13,2,2014-01-01 23:26:00,59.0,48.0,36.50,41.75,...,6.25,9.75,1.25,11.5,0.0,1.25,324.0,False,2.0,1.0
5,1,1,2014,16,2,2014-01-01 17:38:00,59.0,48.0,36.50,41.75,...,6.25,9.75,1.25,11.5,0.0,1.25,324.0,False,2.0,2.0
6,1,1,2014,28,1,2014-01-01 13:58:00,59.0,48.0,36.50,41.75,...,6.25,9.75,1.25,11.5,0.0,1.25,324.0,False,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42831,12,31,2014,76,18,2014-12-31 09:00:00,56.5,46.5,36.25,26.75,...,10.00,19.75,6.25,24.5,0.0,0.50,243.5,False,6.0,11.0
42832,12,31,2014,77,13,2014-12-31 08:04:00,56.5,46.5,36.25,26.75,...,10.00,19.75,6.25,24.5,0.0,0.50,243.5,False,18.0,6.0
42833,12,31,2014,80,1,2014-12-31 15:32:00,56.5,46.5,36.25,26.75,...,10.00,19.75,6.25,24.5,0.0,0.50,243.5,False,13.0,18.0
42834,12,31,2014,82,7,2014-12-31 11:31:00,56.5,46.5,36.25,26.75,...,10.00,19.75,6.25,24.5,0.0,0.50,243.5,False,1.0,13.0


In [94]:
train, test = train_test_split(sf_usage, test_size=0.2)

In [97]:
# When doing inference, we should be able to have previous data for this station at previous months.
# This isn't cheating / leaking info because it's just a way of characterizing which station we're using
# And we're still maintaining the train test split.. right?
train_station_data = (
    train.groupby(["month", "day", 'start_station_id']).agg(
        avg_usage=('usage', 'mean'),
        max_usage=('usage', 'max'),
        min_usage=('usage', 'min'),
    )
).reset_index(drop=False)


In [98]:
# features that give properties of each station instead of the station id
weather_agg = {i: (i, "mean") for i in float_cols}
train_df = (
    train.groupby(["month", "day", 'start_station_id']).agg(
        date=("date", "min"),
        **weather_agg
    )
).reset_index(drop=False)

test_df = (
    test.groupby(["month", "day", 'start_station_id']).agg(
        date=("date", "min"),
        **weather_agg
    )
).reset_index(drop=False)

In [99]:
train_df = train_df.merge(train_station_data, on=["month", "day", "start_station_id"])
test_df = test_df.merge(train_station_data, on=["month", "day", "start_station_id"])

In [100]:
train_df = train_df.drop(["start_station_id", "date"], axis=1)
X_tr = train_df[[i for i in train_df.columns if i != "avg_usage"]]
y_tr = train_df["avg_usage"]
test_df = test_df.drop(["start_station_id", "date"], axis=1)
X_te = test_df[[i for i in train_df.columns if i != "avg_usage"]]
y_te = test_df["avg_usage"]

In [101]:
X_tr

,month,day,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,wind_dir_degrees,max_usage,min_usage
0,1,1,54.333333,44.000,33.333333,27.333333,22.333333,16.000,60.666667,42.666667,...,10.000,10.000,15.000,4.333333,18.000,0.0,0.333333,160.000,2,2
1,1,1,54.333333,44.000,33.333333,27.333333,22.333333,16.000,60.666667,42.666667,...,10.000,10.000,15.000,4.333333,18.000,0.0,0.333333,160.000,1,1
2,1,1,59.000000,48.000,36.500000,41.750000,36.500000,32.000,84.250000,65.750000,...,8.250,6.250,9.750,1.250000,11.500,0.0,1.250000,324.000,1,1
3,1,1,56.666667,46.000,34.916667,34.541667,29.416667,24.000,72.458333,54.208333,...,9.125,8.125,12.375,2.791667,14.750,0.0,0.791667,242.000,2,1
4,1,1,59.000000,48.000,36.500000,41.750000,36.500000,32.000,84.250000,65.750000,...,8.250,6.250,9.750,1.250000,11.500,0.0,1.250000,324.000,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22033,12,31,57.625000,47.375,37.500000,35.125000,27.375000,19.375,75.000000,55.375000,...,7.375,5.000,16.875,4.125000,19.875,0.0,2.125000,252.875,18,10
22034,12,31,57.625000,47.375,37.500000,35.125000,27.375000,19.375,75.000000,55.375000,...,7.375,5.000,16.875,4.125000,19.875,0.0,2.125000,252.875,23,13
22035,12,31,56.500000,46.500,36.250000,26.750000,16.500000,6.750,56.500000,37.000000,...,10.000,10.000,19.750,6.250000,24.500,0.0,0.500000,243.500,1,1
22036,12,31,56.500000,46.500,36.250000,26.750000,16.500000,6.750,56.500000,37.000000,...,10.000,10.000,19.750,6.250000,24.500,0.0,0.500000,243.500,7,7


In [102]:
reg = LinearRegression().fit(X_tr, y_tr)

In [103]:
yhat = reg.predict(X_te)

In [104]:
mean_squared_error(y_te, yhat)

0.00036908621432801895